In [1]:
import torch
import torch.nn.functional as F

# Cấu hình kích thước tensor
batch_size = 1
seq_len = 4   # Số token
embed_dim = 8 # Kích thước embedding
num_heads = 2 # Số head

# Khởi tạo dữ liệu đầu vào và trọng số cố định
X = torch.randn(batch_size, seq_len, embed_dim)
W_Q = torch.randn(embed_dim, embed_dim)
W_K = torch.randn(embed_dim, embed_dim)
W_V = torch.randn(embed_dim, embed_dim)

# Tính toán Q, K, V
Q = torch.matmul(X, W_Q)
K = torch.matmul(X, W_K)
V = torch.matmul(X, W_V)

# Attention Scores (QK^T / sqrt(d_k))
d_k = embed_dim // num_heads
attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (d_k ** 0.5)

# Softmax và tính output O = AV
attn_probs = F.softmax(attn_scores, dim=-1)
O = torch.matmul(attn_probs, V)

# Xuất dữ liệu đầu vào và đầu ra
print("Input X:\n", X)
print("Output O:\n", O)

# Lưu dữ liệu vào file để test Verilog
torch.save({'X': X, 'W_Q': W_Q, 'W_K': W_K, 'W_V': W_V, 'O': O}, "mhsa_test_data.pt")


Input X:
 tensor([[[ 2.5332,  1.4746,  0.2156, -0.7995,  0.3618, -0.0289,  2.3315,
          -1.0273],
         [ 0.2759, -1.0833, -0.6440,  1.4793,  1.5543,  1.0468, -1.0747,
          -0.5896],
         [-0.1337, -0.6911,  1.2067, -0.1629,  0.9028,  1.4611,  0.2346,
           1.0455],
         [ 0.5068,  0.5613,  1.2702,  1.5420, -0.0239,  0.3207,  0.7812,
           0.4886]]])
Output O:
 tensor([[[ 5.0583,  4.2198,  0.0427,  1.0598,  2.3957,  3.2729, -3.2363,
          -3.6075],
         [ 3.4573,  2.7388,  0.1742,  0.4614,  1.4425,  2.0240, -1.9500,
          -2.4402],
         [ 4.9930,  4.1262,  0.0449,  1.0298,  2.3621,  3.1847, -3.2461,
          -3.5546],
         [ 0.1933, -2.7778,  0.1795, -1.1162, -0.1549, -3.2131, -3.8746,
           0.5470]]])


In [9]:
import torch

# Load dữ liệu đã lưu từ PyTorch
data = torch.load("mhsa_test_data.pt")

# Cấu hình Fixed-Point (Q8.8)
FIXED_POINT_SCALE = 2**8  # 256
FIXED_POINT_BITS = 16
MAX_VAL = (1 << (FIXED_POINT_BITS - 1)) - 1  # 32767
MIN_VAL = -(1 << (FIXED_POINT_BITS - 1))     # -32768

def to_fixed(val):
    """Chuyển số thực sang fixed-point Q8.8 (16-bit signed bù hai)."""
    fixed_val = int(round(val * FIXED_POINT_SCALE))
    fixed_val = max(min(fixed_val, MAX_VAL), MIN_VAL)
    return f"{fixed_val & 0xFFFF:04x}"

def save_as_hex(tensor, filename):
    """Lưu tensor thành file .hex để Verilog đọc."""
    with open(filename, "w") as f:
        for row in tensor.reshape(-1, tensor.shape[-1]):  # Đảm bảo đúng format
            for val in row:
                f.write(to_fixed(val.item()) + "\n")

# Lưu các ma trận đầu vào và trọng số
save_as_hex(data["X"], "matrix_X.hex")
save_as_hex(data["W_Q"], "matrix_WQ.hex")
save_as_hex(data["W_K"], "matrix_WK.hex")
save_as_hex(data["W_V"], "matrix_WV.hex")
save_as_hex(data["O"], "matrix_O.hex")  # Kết quả mong muốn để kiểm tra



First 10 lines of matrix_A.hex:










